In [1]:
import sys
#sys.path.insert(0, "../../")
from prediksicovidjatim.data.map import MapDataRepo, MapDataPred, MapDataReal
from prediksicovidjatim import util, config, database

database.init()

## Prepare the Data

In [17]:
selected_kabko = "KAB. BANGKALAN"

In [18]:
real_data = MapDataRepo.fetch_real_data(selected_kabko)
len(real_data)

129

In [19]:
kabko = MapDataRepo.get_kabko_full(selected_kabko)
len(kabko.data)

129

In [20]:
kabko.last_outbreak_shift

-17

In [21]:
from prediksicovidjatim.modeling import SeicrdRlcModel

predict_days = 30
mod = SeicrdRlcModel(kabko)
params = kabko.get_params_init(extra_days=predict_days)
model_result = mod.model(**params)
len(model_result.t)

142

In [22]:
pred_data = MapDataPred.from_result(kabko, model_result)
len(pred_data)

159

In [23]:
assert real_data[0].tanggal == pred_data[0].tanggal

In [24]:
assert util.shift_date(real_data[-1].tanggal, predict_days) == pred_data[-1].tanggal

## Update & Append the Feature Layer

In [25]:
from prediksicovidjatim import util, config
first_tanggal = util.parse_date(config.FIRST_TANGGAL)

In [26]:
real_data = MapDataReal.shift(real_data, first_tanggal)
len(real_data)

129

In [27]:
pred_data = MapDataPred.shift(pred_data, first_tanggal)
len(pred_data)

159

In [28]:
real_id = '0c0a54132fea48ac9a28c13be0c83c88'
pred_id = 'dd84e35e48824fc29a908828c207384c'

In [31]:
from prediksicovidjatim.mapping import MapUpdater

updater = MapUpdater(chunk_size=50)

In [32]:
pred_layer = updater.get_layer(pred_id)
updater.save(pred_layer, selected_kabko, pred_data)

In [33]:
real_layer = updater.get_layer(real_id)
updater.save(real_layer, selected_kabko, real_data)